## Paper Experiments

In [ ]:
import benchmarks
import algorithm
import data
import importlib
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
import tabulate
import cloudpickle
from pathlib import Path
import torch

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def score(predicted, test):
    return np.mean((predicted.reshape(-1)-test.reshape(-1))**2)

# 5.1 Lorenz’s chaotic model

In [ ]:
N = 10 # num of control trajectories
T = 100 # num of time points
dataset = np.zeros((N,T))
p = 10 # only for data generation purposes
F = 5 # amount of chaos
horizon = 30
train_T = 70
batch_size = 128
t_year = 70

def eval_SC(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy):
    predictions_SC, _ = benchmarks.SC(X_train_numpy , Y_train_numpy.reshape(-1, 1), X_test_numpy)
    return np.mean((predictions_SC[t_year:]-Y_test_numpy[t_year:])**2)

def eval_KMM(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy):
    predictions_KMM = benchmarks.KMM(X_train_numpy , Y_train_numpy[:,np.newaxis], X_test_numpy)
    return np.mean((predictions_KMM[t_year:]-Y_test_numpy[t_year:])**2)

def eval_RSC(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy):
    predictions_RSC, _ = benchmarks.elastic_net(X_train_numpy , Y_train_numpy, X_test_numpy)

    return np.mean((predictions_RSC[t_year:]-Y_test_numpy[t_year:])**2)

def eval_NC_SC(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy):
    train_X, train_y = np.expand_dims(X_train_numpy, axis=0), np.expand_dims(Y_train_numpy, axis=0)
    test_X, test_y = np.expand_dims(X_test_numpy, axis=0), np.expand_dims(Y_test_numpy, axis=0)

    train_y = np.expand_dims(train_y, -1)
    test_y = np.expand_dims(test_y, -1)
    train_X, train_y = torch.from_numpy(train_X).float(), torch.from_numpy(train_y).float()
    test_X, test_y = torch.from_numpy(test_X).float(), torch.from_numpy(test_y).float()


    model = algorithm.NeuralCDE(input_channels=train_X.shape[2], hidden_channels=5)

    # l1_reg = 0.01
    iterations = 500

    algorithm.train(model,train_X, train_y, test_X, test_y, iterations)

    predictions_NC_SC = algorithm.predict(model,test_X).squeeze().numpy()
    return np.mean((predictions_NC_SC[t_year:]-Y_test_numpy[t_year:])**2)


# Simulate data
# Only take first dimension of multiple different lorenz models
for i in range(N):
    temp, _ = data.simulate_lorenz_96(p, T=T, F=F, delta_t=0.1)
    temp = np.transpose(temp)
    temp = temp / 8
    dataset[i,:] = temp[1,:]

# torch data
t = torch.linspace(0., 1., dataset.shape[1])
xs = torch.tensor(dataset[1:,:]).float().t().unsqueeze(0)
X = torch.cat([t.unsqueeze(0).unsqueeze(2), xs], dim=2)
Y = torch.tensor(dataset[0,:]).float().unsqueeze(0).unsqueeze(2)

# train data
train_X = torch.zeros(batch_size,horizon,10)
train_y = torch.zeros(batch_size,horizon,1)

for i in range(batch_size):
    index = torch.from_numpy(np.random.choice(np.arange(train_T - horizon, dtype=np.int64), 1, replace=False))
    train_X[i,:,:] = X[:,index:index+horizon,:]
    train_y[i,:,:] = Y[:,index:index+horizon,:]

# test data
test_X, test_y =  X, Y

X_train_numpy, Y_train_numpy = np.transpose(dataset[1:,:t_year]), np.transpose(dataset[0,:t_year])
X_test_numpy, Y_test_numpy = np.transpose(dataset[1:,:]), np.transpose(dataset[0,:])

score_SC_lorenz = eval_SC(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy)
score_KMM_lorenz = eval_KMM(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy)
score_RSC_lorenz = eval_RSC(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy)
score_NC_SC_lorenz = eval_NC_SC(X_train_numpy, Y_train_numpy, X_test_numpy, Y_test_numpy)

headers = ["Method", "Score"]
results = [
    ["SC", score_SC_lorenz],
    ["KMM", score_KMM_lorenz],
    ["R-SC", score_RSC_lorenz],
    ["NC-SC", score_NC_SC_lorenz],
    
]
display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

# 5.2 The Eurozone and current account deficits

In [ ]:
import data

# import data
X , Y = data.get_emu_data_numpy()
t_year = 19
X_train , Y_train = X[:t_year].copy(), Y[:t_year,np.newaxis].copy()
X_test , Y_test = data.get_emu_data_numpy()

In [ ]:
predictions_SC, w_SC = benchmarks.SC(X_train , Y_train, X_test)
score_SC = np.mean((predictions_SC[:t_year]-Y_test[:t_year])**2)

In [ ]:
predictions_KMM = benchmarks.KMM(X_train , Y_train, X_test)
score_KMM = np.mean((predictions_KMM[:t_year]-Y_test[:t_year])**2)

In [ ]:
predictions_elastic_net, _ = benchmarks.elastic_net(X_train , Y_train, X_test)
score_elastic_net = np.mean((predictions_elastic_net[:t_year]-Y_test[:t_year])**2)

In [ ]:
X, Y = data.get_emu_data()
train_X, train_y = X[:,:t_year,:], Y[:,:t_year,:]
test_X, test_y = data.get_emu_data()
model_path = Path("eurozone_model.p")

if not model_path.exists():
    model = algorithm.NeuralCDE(input_channels=train_X.shape[2], hidden_channels=5)
    
    # l1_reg = 0.01
    iterations = 1000

    algorithm.train(model,train_X, train_y, test_X, test_y, iterations)

    with open(model_path, "wb") as f:
        cloudpickle.dump(model, f)
else:
    with open(model_path, "rb") as f:
        model = cloudpickle.load(f)

predictions_NC_SC = algorithm.predict(model,test_X).squeeze().numpy()
score_NC_SC = np.mean((predictions_NC_SC[:t_year]-Y_test[:t_year])**2)

### Eurozone Counterfactual estimation performance.

In [ ]:
headers = ["Method", "Score"]
results = [
    ["SC", score_SC],
    ["KMM", score_KMM],
    ["R-SC", score_elastic_net],
    ["NC-SC", score_NC_SC],
    
]
display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)


# 5.3 Smoking control in California

In [ ]:
# import data in correct format
t_year = 19
X , Y = data.get_smoking_data_numpy()
X_train , Y_train = X[:t_year], Y[:t_year,np.newaxis]
X_test , Y_test = data.get_smoking_data_numpy()

In [ ]:
predictions_SC, w_SC = benchmarks.SC(X_train , Y_train, X_test)
score_SC_smoke = np.mean((predictions_SC[:t_year]-Y_test[:t_year])**2)

In [ ]:
predictions_KMM = benchmarks.KMM(X_train , Y_train, X_test)
score_KMM_smoke = np.mean((predictions_KMM[:t_year]-Y_test[:t_year])**2)

In [ ]:
predictions_elastic_net, _ = benchmarks.elastic_net(X_train , Y_train, X_test)
score_elastic_net_smoke = np.mean((predictions_elastic_net[:t_year]-Y_test[:t_year])**2)

In [ ]:
predictions_mc = benchmarks.MC_NNM(X_train , Y_train.reshape(-1, 1), X_test, Y_test.reshape(-1, 1))
score_mc_smoke = score_mc_smoke = np.mean((predictions_mc[:t_year]-Y_test[:t_year])**2)

In [ ]:
X, Y = data.get_smoking_data()
train_X, train_y = X[:,:t_year,:], Y[:,:t_year,:]
test_X, test_y = data.get_smoking_data()

model_path = Path("smoking_model.p")

if not model_path.exists():
    model = algorithm.NeuralCDE(input_channels=train_X.shape[2], hidden_channels=5)
    
    # l1_reg = 0.01
    iterations = 1000

    algorithm.train(model,train_X, train_y, test_X, test_y, iterations)

    with open(model_path, "wb") as f:
        cloudpickle.dump(model, f)
else:
    with open(model_path, "rb") as f:
        model = cloudpickle.load(f)

predictions_NC_SC = algorithm.predict(model,test_X).squeeze().numpy()
score_NC_SC_smoke = np.mean((predictions_NC_SC[:t_year]-Y_test[:t_year])**2)

###  Smoking control Counterfactual estimation performance.

In [ ]:
headers = ["Method", "Score"]
results = [
    ["SC", score_SC_smoke],
    ["KMM", score_KMM_smoke],
    ["R-SC", score_elastic_net_smoke],
    ["MC-NNM", score_mc_smoke],
    ["NC-SC", score_NC_SC_smoke],
    
]
display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)
